In [1]:
from selenium import webdriver
from time import sleep
import threading
import re
from tqdm import tnrange, tqdm_notebook
import urllib 

In [2]:
items = pd.read_csv('data/item.csv')
items_detail= pd.read_csv('data/item_detail.csv')

In [3]:
len(items), len(items_detail)

(5180, 5180)

In [4]:
merged = pd.merge(items, items_detail, how='inner', on='id_item')

In [5]:
def getname(name, brand): 
    return name.replace(str(brand), '')

In [6]:
merged['item_name'] = merged.apply(lambda row: getname(row['name'], row['brand']), axis=1)

In [175]:
def get_comment(id_item, keyword, driver):
     
    ukey = urllib.parse.quote(keyword.encode('utf8'), '/:') 
    
    url = "https://msearch.shopping.naver.com/search/all.nhn?origQuery={}&pagingIndex=1&viewType=lst&sort=rel&showFilter=true&frm=NVSHSRC&selectedFilterTab=price&query={}".format(ukey,ukey)
       
    driver.get(url) 
    sleep(1)
     
    reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .grade') 
    rows = []
    if len(reviews) > 0: 
        for review in reviews:
            
            parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
            link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
            num = review.find_element_by_css_selector('.num').text
            cmts = review.find_element_by_css_selector('.help').text
            
            rows.append([id_item, link, num, cmts, keyword])
    else :
        reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .piece') 
        
        revcnt = 0
        
        for review in reviews:
            if review.text.startswith('리뷰'): 
                cmts = review.text.split('리뷰')[1] 
                revcnt += 1 
                parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
                link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
                #print(link)
                rows.append([id_item, link, np.nan, cmts, keyword])
        
        if revcnt == 0:
            rows.append([id_item, np.nan, np.nan, np.nan, keyword])
    return rows
    
def get_comment_group(items, names):
    driver =  webdriver.PhantomJS()
    for item, name in zip(items, names): 
         
        review_links = get_comment(item, name, driver)
        items_comments.extend(review_links)
    driver.close() 

In [176]:
!killall phantomjs

No matching processes belonging to you were found


In [177]:
items_comments = []
for cnt in range(1, 11): 
    start = 518 * (cnt-1)
    end = 518 * cnt-1
    items = merged.loc[start:end]['id_item'].values
    f_names = merged.loc[start:end]['fullname'].values
  
    th = threading.Thread(target=get_comment_group, args=(items,f_names, ))
    th.start()

In [197]:
review_info = pd.DataFrame(items_comments,columns=['id_item','link','point','cnt','keyword'])

In [199]:
!ls data

item.csv        item_detail.csv


In [200]:
review_info.to_csv('data/review_info.csv',index=False)

In [47]:
review_info = pd.read_csv('data/review_info.csv', thousands=',')

In [60]:
n_review_info = review_info[review_info['point'].notna()]
n_review_info = n_review_info.reset_index(drop=True)
n_review_info['cnt'] = n_review_info['cnt'].astype(int)

In [61]:
len(n_review_info)

2471

In [74]:
2471 / 7

353.0

In [117]:
def get_naver_review(url, cnt, iid, driver):
    driver.get(url) 
    lis = driver.find_elements_by_css_selector('ul.lst_review li:not(.u_likeit_list)')
     
    
    rows = []
    for page_seq in range(2, cnt//2 + 1):
        for li in lis: 
            cls = li.get_attribute('class').strip()  
            point = li.find_element_by_css_selector('.atc_area .avg_area .curr_avg').text
            site = li.find_element_by_css_selector('.atc_area .avg_area .info .path').text
            person = li.find_element_by_css_selector('.atc_area .avg_area .info .name').text

            date = li.find_element_by_css_selector('.atc_area .avg_area .date').text
            subj = li.find_element_by_css_selector('.atc_area .subjcet').text 
            atc = li.find_element_by_css_selector('div.atc_area .atc').text
             
            rows.append([iid, point, site, person, date, subj, atc]) 
       
        # 페이지 갱신
        driver.execute_script("shop.detail.ReviewHandler.page({}, '_review_paging');".format(page_seq))
        sleep(0.3)
        lis = driver.find_elements_by_css_selector('ul.lst_review li:not(.u_likeit_list)')

    return rows


def get_naver_review_group(links, cnts, ids):
    
    driver =  webdriver.PhantomJS()
    for link, cnt, iid in zip(links, cnts, ids):  
        naver_reviews = get_naver_review(link, cnt, iid, driver)
        all_naver_reviews.extend(naver_reviews)
    driver.close() 

In [161]:
!killall phantomjs

No matching processes belonging to you were found


In [119]:
all_naver_reviews = []
for cnt in range(1, 8): 
    start = 353 * (cnt-1)
    end = 353 * cnt
    links = n_review_info.loc[start:end]['link'].values
    cnts = n_review_info.loc[start:end]['cnt'].values
    ids = n_review_info.loc[start:end]['id_item'].values
  
    print(start, end)
    th = threading.Thread(target=get_naver_review_group, args=(links, cnts, ids ))
    th.start()

0 353
353 706
706 1059
1059 1412
1412 1765
1765 2118
2118 2471


In [158]:
len(all_naver_reviews)

188

In [159]:
pd.DataFrame(all_naver_reviews)

,0,1,2,3,4,5,6
0,28163,5,신세계몰,gr02****님,2013.11.18.,사진 그대로에요. 아직 써보진 않았지만 좋다고 해서 기대는 됩니다.,신세계몰에서 한번에 몰아서 주문했는데 다 따로 분리되서 배송되었습니다.\n택배박스만...
1,28163,5,신세계몰,coco****님,2012.12.10.,전 신세계 매장에서 주로 구입을 많이해요,블리스제품을 사용 해보고 좋은상품이 많은듯하여\n이곳에 글을 남기네요\n토너는 저처...
2,28163,5,신세계몰,onen****님,2012.11.30.,겨울이라 가벼운 토너보다 보습력이 좋은 토너 찾고 있었는데 이 제품 너무 좋더라구요,전에 에스티 토너보다 느낌은 산뜻한데 보습은 더 좋은듯 해요\n저랑 동생이 트러블이...
3,28163,5,현대백화점,mapl****님,2015.01.05.,빠른 배송 감사합니다,빠른 배송 감사합니다
4,28163,5,현대백화점,yn04****님,2013.12.04.,저렴하게 구매했네요,저렴하게 구매했네요
5,28163,5,현대백화점,aros****님,2013.12.01.,잘받았어요,잘받았어요
6,28163,5,신세계몰,rlfq****님,2013.10.24.,[블리스]데일리 페이셜 토너 200ml,블리스 제품 처음 괜찮아요~~\n[블리스]데일리 페이셜 토너 200ml\n블리스 제...
7,28163,5,신세계몰,hhan****님,2013.10.16.,포장 상태는 싸구려 제품 포장 상태이네요,포장 상태는 싸구려 제품 포장 상태이네요
8,28163,5,신세계몰,lind****님,2013.07.22.,매우 만족하는 제품...,매우 만족하는 제품...
9,28163,5,신세계몰,dora****님,2013.04.01.,전 오프라인 롯데백화점에서 샀는데 이 스킨 진짜 딱 효과 좋아요. 다른 스킨들은 그...,전 오프라인 롯데백화점에서 샀는데 이 스킨 진짜 딱 효과 좋아요. 다른 스킨들은 그...
